## LASSO Forecast

Esse notebook tem como objetivo estimar a variável de previsão a partir do LASSO, $f_{n, t}^{\mathrm{LASSO}}$.

$$
f_{n, t}^{\mathrm{LASSO}} \stackrel{\text { def }}{=} \tilde{\alpha}_n+\sum_{n^{\prime}=1}^{3 \cdot N} \tilde{\beta}_{n, n^{\prime}} \cdot x_{n^{\prime}, t}
$$

onde $x_{n^{\prime}, t}$ é o retorno do ativo $n^{\prime}$ padronizado para ter média nula e variância unitária dentro da janela de estimação de 30 minutos.

In [25]:
# pacotes
import numpy as np
import pandas as pd
import statsmodels.formula.api as sm

In [2]:
# oculta mensagens de avisos
import warnings
warnings.filterwarnings("ignore")

In [3]:
x = pd.read_csv('../../output/data/20030102_x.csv', index_col=0)

In [9]:
x = x[3:]

In [10]:
x

,A(t),AA(t),AAAB(t),AAC(t),AACE(t),AAI(t),AAII(t),AAME(t),AANB(t),AAON(t),...,ZOOM(t-3),ZQK(t-1),ZQK(t-2),ZQK(t-3),ZRAN(t-1),ZRAN(t-2),ZRAN(t-3),ZTEL(t-1),ZTEL(t-2),ZTEL(t-3)
Time,,,,,,,,,,,,,,,,,,,,,
93400,-0.001091,0.000865,0.0,0.0,0.0,0.000000,0.001403,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,-0.002073,-0.002068,0.016667,0.0,0.0,-0.140357
93500,0.000000,0.000000,0.0,0.0,0.0,0.000000,-0.001403,0.0,0.0,0.000000,...,0.0,0.000371,0.000000,0.000000,0.005724,-0.002073,-0.002068,0.0,0.0,0.000000
93600,0.001635,-0.002599,0.0,0.0,0.0,0.000000,-0.003515,0.0,0.0,0.000000,...,0.0,0.000370,0.000371,0.000000,-0.007800,0.005724,-0.002073,0.0,0.0,0.000000
93700,0.000545,-0.002171,0.0,0.0,0.0,0.002516,0.000000,0.0,0.0,0.000000,...,0.0,0.003697,0.000370,0.000371,-0.004863,-0.007800,0.005724,0.0,0.0,0.000000
93800,-0.006006,0.002387,0.0,0.0,0.0,0.000000,0.003515,0.0,0.0,0.000000,...,0.0,0.000000,0.003697,0.000370,-0.000871,-0.004863,-0.007800,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155600,0.000000,-0.000424,0.0,0.0,0.0,0.002378,-0.003503,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,-0.000596,0.001923,0.000000,0.0,0.0,0.000000
155700,-0.001043,0.000000,0.0,0.0,0.0,-0.002378,0.001403,0.0,0.0,0.000000,...,0.0,0.001096,0.000000,0.000000,0.001655,-0.000596,0.001923,0.0,0.0,0.000000
155800,0.000000,-0.000848,0.0,0.0,0.0,0.000000,-0.000701,0.0,0.0,0.000000,...,0.0,0.000730,0.001096,0.000000,-0.001655,0.001655,-0.000596,0.0,0.0,0.000000


In [11]:
y = pd.read_csv('../../output/data/20030102_y.csv', index_col=0)

In [13]:
y = y[3:]

In [14]:
y

,IFUL(t),RMD(t),NI(t),HYSQ(t),HSC(t),ACDO(t),GNLB(t),DRVR(t),BJCT(t),SP(t),...,BPRX(t),DLX(t),RRGB(t),PLUM(t),CALA(t),DHB(t),RRA(t),RMHT(t),FDTR(t),DRRX(t)
Time,,,,,,,,,,,,,,,,,,,,,
93400,0.0,0.000000,0.000499,0.000000,0.000313,-0.000568,0.005731,0.0,0.0,0.003069,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,-0.004728
93500,0.0,0.000000,0.000000,0.000000,0.000000,-0.001421,-0.028988,0.0,0.0,0.000000,...,0.000000,-0.000953,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
93600,0.0,0.000000,0.000998,0.000000,0.000000,-0.001708,0.000000,0.0,0.0,-0.001022,...,0.000000,0.000477,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
93700,0.0,0.000000,-0.002497,0.000000,0.000313,-0.003426,0.000000,0.0,0.0,0.004082,...,0.000000,0.000238,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
93800,0.0,0.000000,0.002497,0.000000,0.000000,-0.003437,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.004728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155600,0.0,0.000000,-0.001956,-0.000602,0.000000,-0.001103,0.000000,0.0,0.0,-0.001043,...,0.000000,0.000238,-0.000039,0.000000,0.0,0.000000,0.002732,0.000000,0.0,0.000000
155700,0.0,0.000322,0.000489,0.000000,0.001547,0.000000,0.000000,0.0,0.0,-0.001044,...,0.003656,0.000000,0.000039,0.000000,0.0,0.000000,0.004084,0.007737,0.0,0.000000
155800,0.0,0.000000,-0.000979,0.000000,0.000927,0.000000,0.000000,0.0,0.0,0.000000,...,-0.001461,-0.000238,0.004294,0.003824,0.0,0.000000,0.000000,-0.007737,0.0,0.000000


In [52]:
x_temp = x[:30]

In [53]:
x_temp = ( x_temp - x_temp.mean() ) / x_temp.std()

In [58]:
x_temp.dropna(axis=1)

,A(t),AA(t),AAI(t),AAII(t),AANB(t),AAP(t),AAPL(t),ABC(t),ABCB(t),ABCO(t),...,ZONA(t-1),ZONA(t-2),ZONA(t-3),ZQK(t-1),ZQK(t-2),ZQK(t-3),ZRAN(t-1),ZRAN(t-2),ZRAN(t-3),ZTEL(t-3)
Time,,,,,,,,,,,,,,,,,,,,,
93400,-0.496731,0.354248,0.128370,0.960861,0.078206,0.890442,-1.407870,-0.103034,0.182574,-0.168453,...,-1.296188e-15,-1.296188e-15,0.070999,0.023407,0.125689,0.125689,-0.456867,-0.421561,3.705356,-5.294651
93500,0.045198,-0.139421,0.128370,-0.734074,0.078206,0.888614,-0.448033,-0.649496,0.182574,-0.435318,...,-1.296188e-15,-1.296188e-15,0.070999,0.257303,0.125689,0.125689,1.929727,-0.422878,-0.419270,0.182574
93600,0.857870,-1.621712,0.128370,-2.010489,0.078206,0.526255,0.177301,-2.017992,0.182574,-0.168453,...,-1.296188e-15,-1.296188e-15,0.070999,0.257216,0.394272,0.125689,-2.210319,1.971974,-0.420214,0.182574
93700,0.315793,-1.377613,2.046653,0.113394,0.078206,-1.008084,-0.752906,-0.559514,0.182574,-0.168453,...,-1.296188e-15,-1.296188e-15,0.070999,2.356750,0.394173,0.394272,-1.311043,-2.182397,1.296118,0.182574
93800,-2.939489,1.222442,0.128370,2.237276,0.078206,-1.826186,-0.127351,-0.833833,0.182574,-3.377816,...,-1.296188e-15,-1.296188e-15,0.070999,0.023407,2.805072,0.394173,-0.088998,-1.280009,-1.681218,0.182574
93900,1.132614,-0.510516,-1.789913,0.113394,0.078206,-0.015100,-0.127351,0.079715,0.182574,-0.168453,...,-1.296188e-15,-1.296188e-15,0.070999,-1.842577,0.125689,2.805072,-2.124726,-0.053737,-1.034499,0.182574
94000,1.401130,1.096624,0.128370,0.113394,0.078206,0.347704,0.193551,1.540220,0.182574,-0.168453,...,-1.296188e-15,-1.296188e-15,0.070999,-2.551361,-2.017026,0.125689,0.821844,-2.096508,-0.155659,0.182574
94100,-2.126073,-1.375466,2.046653,0.113394,0.078206,0.256810,-0.127351,1.719662,0.182574,0.367338,...,-1.296188e-15,-1.296188e-15,0.070999,0.023407,-2.830925,-2.017026,0.820491,0.860257,-1.619663,0.182574
94200,0.045198,-0.634590,0.128370,-2.010489,0.078206,-0.015100,0.193330,-2.838317,0.182574,-0.168453,...,-1.296188e-15,-1.296188e-15,0.070999,-0.211185,0.125689,-2.830925,0.819144,0.858899,0.499378,0.182574


In [57]:
x_temp

,A(t),AA(t),AAAB(t),AAC(t),AACE(t),AAI(t),AAII(t),AAME(t),AANB(t),AAON(t),...,ZOOM(t-3),ZQK(t-1),ZQK(t-2),ZQK(t-3),ZRAN(t-1),ZRAN(t-2),ZRAN(t-3),ZTEL(t-1),ZTEL(t-2),ZTEL(t-3)
Time,,,,,,,,,,,,,,,,,,,,,
93400,-0.496731,0.354248,NaN,NaN,NaN,0.128370,0.960861,NaN,0.078206,NaN,...,NaN,0.023407,0.125689,0.125689,-0.456867,-0.421561,3.705356,NaN,NaN,-5.294651
93500,0.045198,-0.139421,NaN,NaN,NaN,0.128370,-0.734074,NaN,0.078206,NaN,...,NaN,0.257303,0.125689,0.125689,1.929727,-0.422878,-0.419270,NaN,NaN,0.182574
93600,0.857870,-1.621712,NaN,NaN,NaN,0.128370,-2.010489,NaN,0.078206,NaN,...,NaN,0.257216,0.394272,0.125689,-2.210319,1.971974,-0.420214,NaN,NaN,0.182574
93700,0.315793,-1.377613,NaN,NaN,NaN,2.046653,0.113394,NaN,0.078206,NaN,...,NaN,2.356750,0.394173,0.394272,-1.311043,-2.182397,1.296118,NaN,NaN,0.182574
93800,-2.939489,1.222442,NaN,NaN,NaN,0.128370,2.237276,NaN,0.078206,NaN,...,NaN,0.023407,2.805072,0.394173,-0.088998,-1.280009,-1.681218,NaN,NaN,0.182574
93900,1.132614,-0.510516,NaN,NaN,NaN,-1.789913,0.113394,NaN,0.078206,NaN,...,NaN,-1.842577,0.125689,2.805072,-2.124726,-0.053737,-1.034499,NaN,NaN,0.182574
94000,1.401130,1.096624,NaN,NaN,NaN,0.128370,0.113394,NaN,0.078206,NaN,...,NaN,-2.551361,-2.017026,0.125689,0.821844,-2.096508,-0.155659,NaN,NaN,0.182574
94100,-2.126073,-1.375466,NaN,NaN,NaN,2.046653,0.113394,NaN,0.078206,NaN,...,NaN,0.023407,-2.830925,-2.017026,0.820491,0.860257,-1.619663,NaN,NaN,0.182574
94200,0.045198,-0.634590,NaN,NaN,NaN,0.128370,-2.010489,NaN,0.078206,NaN,...,NaN,-0.211185,0.125689,-2.830925,0.819144,0.858899,0.499378,NaN,NaN,0.182574


In [ ]:
def temporary_standardization(x, t):
    x_temp = x[t:30+t]
    x_temp = ( x_temp - x_temp.mean() ) / x_temp.std()

In [10]:
df.iloc[:,0] - df.iloc[:,0].mean()

,A(t),AA(t),AAAB(t),AAC(t),AACE(t),AAI(t),AAII(t),AAME(t),AANB(t),AAON(t),...,ZOOM(t-3),ZQK(t-1),ZQK(t-2),ZQK(t-3),ZRAN(t-1),ZRAN(t-2),ZRAN(t-3),ZTEL(t-1),ZTEL(t-2),ZTEL(t-3)
Time,,,,,,,,,,,,,,,,,,,,,
93100,0.000000,0.002602,0.0,0.0,0.0,0.017790,-0.000702,0.0,0.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93200,0.001645,-0.001734,0.0,0.0,0.0,0.000000,0.002735,0.0,0.0,0.000000,...,NaN,0.000000,NaN,NaN,0.016667,NaN,NaN,-0.140357,NaN,NaN
93300,0.005464,0.002167,0.0,0.0,0.0,0.000000,-0.002033,0.0,0.0,0.000000,...,NaN,0.000000,0.000000,NaN,-0.002068,0.016667,NaN,0.000000,-0.140357,NaN
93400,-0.001091,0.000865,0.0,0.0,0.0,0.000000,0.001403,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,-0.002073,-0.002068,0.016667,0.000000,0.000000,-0.140357
93500,0.000000,0.000000,0.0,0.0,0.0,0.000000,-0.001403,0.0,0.0,0.000000,...,0.0,0.000371,0.000000,0.000000,0.005724,-0.002073,-0.002068,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155600,0.000000,-0.000424,0.0,0.0,0.0,0.002378,-0.003503,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,-0.000596,0.001923,0.000000,0.000000,0.000000,0.000000
155700,-0.001043,0.000000,0.0,0.0,0.0,-0.002378,0.001403,0.0,0.0,0.000000,...,0.0,0.001096,0.000000,0.000000,0.001655,-0.000596,0.001923,0.000000,0.000000,0.000000
155800,0.000000,-0.000848,0.0,0.0,0.0,0.000000,-0.000701,0.0,0.0,0.000000,...,0.0,0.000730,0.001096,0.000000,-0.001655,0.001655,-0.000596,0.000000,0.000000,0.000000
